# Conections invariant under the involution

Let $C$ be the elliptic curve is given by $y^2 = P(x)$, where $P(x) = x(x-1)(x-\lambda)$. We consider here connections on the trivial bundle $\mathcal{O}_C\oplus\mathcal{O}_C$ having two logarithmic singularities over two points in involution $(t,r), (t,-r)$ and two apparent singularities over another pair of points in involution $(p,q),(p,-q)$.  

In [1]:
import re
from IPython.core.display import display, HTML, Math

# Define the width of the cells
display(HTML("<style>.container { width:85% !important; }</style>"))

def DM(expression):
    """Short for DisplayMath: converts symbolic expression into latex and displays it nicely"""
    return display(Math(latex(expression)))

In [2]:
# We use the following variables:
var('x y z L t r p q u n a_0 a_1 a_2 b_0 b_1 b_2 b_3 c1 c2')
variables = [x, y, L, t, r, p, q, u, n]
parameters = [a_0, a_1, a_2, b_0, b_1, b_2, b_3]

| Variable | Meaning |
| :---: | --- |
| `(x,y)` | Coordinates on $\mathbb{C}^2$ |
| `P` | The polynomial $x(x-1)(x-\lambda)$ defining the curve $C$ |
| `z` | Coordinate on the (projectivized) fiber is $[1:z]$ |
| `L` | The number $\lambda$ defining the curve |
| `(t, +-r)` | The coordinates of the points over $x=t$ |
| `(p, +-q)` | The coordinates of the points over $x=p$ |
| `u` | $z$-coordinate of the parabolic point over $(t,r)$ |
| `n` | Stands for $\eta$, the exponent of the singularity over $(t,r)$ |
| `a_i, b_i` | Parameters defining a connection |
| `c1, c2` | Coefficients used to combine $\nabla_0$, $\Theta_1$ and $\Theta_2$ |
| `k1, k2` | Coefficients used to combine $\hat\nabla_0$, $\hat\Theta_1$ and $\hat\Theta_2$ |

A connection in our family is given as

$$\nabla = d + \frac{M}{(x-p)(x-t)}\frac{dx}{y}, \quad M=\pmatrix{
    \alpha & \beta \\
    \gamma & -\alpha \\
},$$

where $\alpha = a_0 + a_1x + a_2x^2 + a_3y$, and so on.

## Normal form for our family of connections

We choose to use the following normal form:  
`a_3 = 0, c_3 = b_3, c_2 = -b_2, c_1 = -b_1, c_0 = -b_0.` 

This means that the connection $\nabla$ is invariant by the following transformation: Pull back $\nabla$ by the elliptic involution and exchange the order of the standard basis of $\mathcal{O}_C\oplus\mathcal{O}_C$.  
In particular, we have the following: By definition `u` is the height of the parabolic over $(t,r)$, then in our normal form the parabolic over $(t,-r)$ is located at `1/u`.

In [3]:
a_3=0
c_3=b_3
c_2=-b_2
c_1=-b_1
c_0=-b_0

In [4]:
a = a_0 + a_1*x + a_2*x^2 + a_3*y
b = b_0 + b_1*x + b_2*x^2 + b_3*y
c = c_0 + c_1*x + c_2*x^2 + c_3*y

In [5]:
M = matrix([[a, b], [c, -a]])
P = x*(x-1)*(x-L)
dP = diff(P, x)

### Auxiliary function `simplify_qr`

I will use this function throughout the notebook to simplify powers of $q = \sqrt{p(p-1)(p-\lambda)}$ and $r = \sqrt{t(t-1)(t-\lambda)}$.

In [168]:
# Auxiliary function to simplify powers of q and r (up to degree 4)

q2 = P.subs(x==p)
r2 = P.subs(x==t)

# Create dictionaries to pass onto the 'sage_eval' command
dict_vars = {str(variable): variable for variable in variables}
dict_vars['r2'] = r2
dict_vars['q2']  = q2

dict_params = dict_vars.copy()
dict_params.update({str(param): param for param in parameters})

def simplify_qr(expr, dictionary=dict_vars):
    """Convert expr to a string and replace powers of q and r"""
    expr = expr.simplify_rational()
    num = expr.numerator().expand()
    str_num = str(num)
    str_num = re.sub('q\^2', 'q2', str_num)
    str_num = re.sub('q\^3', 'q*q2', str_num)
    str_num = re.sub('q\^4', 'q2^2', str_num)
    str_num = re.sub('r\^2', 'r2', str_num)
    str_num = re.sub('r\^3', 'r*r2', str_num)
    str_num = re.sub('r\^4', 'r2^2', str_num)
    
    den = expr.denominator().expand()
    str_den = str(den)
    str_den = re.sub('q\^2', 'q2', str_den)
    str_den = re.sub('q\^3', 'q*q2', str_den)
    str_den = re.sub('q\^4', 'q2^2', str_den)
    str_den = re.sub('r\^2', 'r2', str_den)
    str_den = re.sub('r\^3', 'r*r2', str_den)
    str_den = re.sub('r\^4', 'r2^2', str_den)
    
    new_expr = sage_eval(str_num, locals=dictionary)/sage_eval(str_den, locals=dictionary)
    try:
        new_expr = new_expr.simplify_rational()
    except AttributeError:
        return new_expr
    try:
        new_expr = new_expr.factor()
    except ArithmeticError:
        return new_expr
    
    return new_expr

## Computation of the residues

The residues are given by the following formulas:

$\operatorname{Res}_{p_i}\nabla = \operatorname{Res}_{p_i} \frac{M}{y(x-t)} \frac{dx}{x-p} = \frac{M(p_i)}{\pm q(p-t)}$

$\operatorname{Res}_{t_i}\nabla = \operatorname{Res}_{t_i} \frac{M}{y(x-p)} \frac{dx}{x-t} = \frac{M(t_i)}{\pm r(t-p)}$

In [14]:
Sing = [[x==p, y==q], [x==p, y==-q], [x==t, y==r], [x==t, y==-r]]

In [15]:
Res = ['']*4

Res[0] = (M/(y*(x-t))).subs(Sing[0])
Res[1] = (M/(y*(x-t))).subs(Sing[1])
Res[2] = (M/(y*(x-p))).subs(Sing[2])
Res[3] = (M/(y*(x-p))).subs(Sing[3])

## Linear conditions on the type of singularities

The parameters `a_i, b_i` are subject to the following constrains:

* The residue at $p_1$ has `(0,1)` as a parabolic direction with eigenvalue `1/2`,
* The residue at $t_1$ has `(1,u)` as parabolic direction with eigenvalue `n/2`.

By symmetry, this also fixes the type of singularities over $p_2$ and $t_2$.

In [16]:
# We get 5 equations besides the invariance conditions
Eq = [0]*5

Parabolic direction and eigenvalue over $p_1$:

In [17]:
Eq[0] = Res[0][0][1].numerator()
Eq[1] = (Res[0][1][1] - 1/2).simplify_rational().numerator()

Parabolic direction and eigenvalue over $t_1$:

In [18]:
V1 = Res[2]*vector([1, u]) - (n/2)*vector([1, u])

In [19]:
Eq[2] = V1[0].simplify_rational().numerator()
Eq[3] = V1[1].simplify_rational().numerator()

There is one more linear condition that guarantees that the singularity over $p_1$ is apparent. To find it, let us expand $y=\sqrt{P(x)}$ as a power series near $p_1$:  
$ y(x) = q + \frac{1}{2q}P'(p)(x-p) + \ldots$

In [20]:
# reacall: P = x*(x-1)*(x-L)
l = diff(P, x).subs(x==p).expand()
y_truncated = q + l*(x-p)/(2*q)

In [21]:
C01 = (M[0][1]/(y*(x-t))).subs(y==y_truncated)

In [22]:
constant_term = diff(C01, x).subs(x==p)

In [23]:
# Simplify q^2 and q^3 using 'simplify_qr'
Eq[4] = simplify_qr(constant_term, dictionary=dict_params).numerator() 

### The five linear conditions:

In [24]:
for e in Eq:
    print e.factor(), '\n'

b_2*p^2 + b_1*p + b_3*q + b_0 

-2*a_2*p^2 - 2*a_1*p - p*q + q*t - 2*a_0 

-2*b_2*t^2*u - n*p*r + n*r*t - 2*a_2*t^2 - 2*b_3*r*u - 2*b_1*t*u - 2*a_1*t - 2*b_0*u - 2*a_0 

-n*p*r*u + n*r*t*u + 2*a_2*t^2*u + 2*b_2*t^2 + 2*a_1*t*u - 2*b_3*r + 2*b_1*t + 2*a_0*u + 2*b_0 

b_2*p^5 - 2*L*b_2*p^3*t + b_2*p^4*t - 2*L*b_1*p^3 - L*b_2*p^3 + 3*b_1*p^4 - 2*L*b_3*p^2*q + 2*b_3*p^3*q + 3*L*b_2*p^2*t - b_1*p^3*t - 2*b_2*p^3*t - 4*L*b_0*p^2 + L*b_1*p^2 + 5*b_0*p^3 - 2*b_1*p^3 + 2*L*b_3*p*q - 2*b_3*p^2*q + 2*L*b_0*p*t + L*b_1*p*t - 3*b_0*p^2*t + 3*L*b_0*p - 4*b_0*p^2 - L*b_0*t + 2*b_0*p*t 



In [25]:
Sols = solve(Eq, [a_0, a_1, b_0, b_1, b_2])[0]  # a_2, b_3 are free

In [26]:
# The values of the parameters in terms of a_2, b_3
for param in parameters:
    if param.subs(Sols) == 0:
        print '{} = 0'.format(param), '\n'
    else:
        print '{} ='.format(param), param.subs(Sols).simplify_rational().factor(), '\n'

a_0 = 1/2*(n*p^2*r*u^2 + 2*a_2*p^2*t*u^2 - n*p*r*t*u^2 - 2*a_2*p*t^2*u^2 + p*q*t*u^2 - q*t^2*u^2 + n*p^2*r - 2*a_2*p^2*t - n*p*r*t + 2*a_2*p*t^2 + 4*b_3*p*r*u - p*q*t + q*t^2)/((p - t)*(u + 1)*(u - 1)) 

a_1 = -1/2*(2*a_2*p^2*u^2 + n*p*r*u^2 - n*r*t*u^2 - 2*a_2*t^2*u^2 + p*q*u^2 - q*t*u^2 - 2*a_2*p^2 + n*p*r - n*r*t + 2*a_2*t^2 + 4*b_3*r*u - p*q + q*t)/((p - t)*(u + 1)*(u - 1)) 

a_2 = a_2 

b_0 = -1/2*(2*L*n*p^4*r*u - 2*n*p^5*r*u - 2*L*n*p^3*r*t*u + 2*n*p^4*r*t*u + 2*L*b_3*p^3*r*u^2 - 2*b_3*p^4*r*u^2 - 2*L*b_3*p^2*q*t*u^2 + b_3*p^3*q*t*u^2 + b_3*p^2*q*t^2*u^2 - 2*L*n*p^3*r*u + 2*n*p^4*r*u + 2*L*n*p^2*r*t*u - 2*n*p^3*r*t*u - 2*L*b_3*p^2*r*u^2 + 2*b_3*p^3*r*u^2 + 3*L*b_3*p*q*t*u^2 - 2*b_3*p^2*q*t*u^2 - L*b_3*q*t^2*u^2 + 2*L*b_3*p^3*r - 2*b_3*p^4*r + 2*L*b_3*p^2*q*t - b_3*p^3*q*t - b_3*p^2*q*t^2 - 2*L*b_3*p^2*r + 2*b_3*p^3*r - 3*L*b_3*p*q*t + 2*b_3*p^2*q*t + L*b_3*q*t^2)/((L - p)*(p - t)^2*(p - 1)*(u + 1)*(u - 1)) 

b_1 = 1/2*(4*L*n*p^4*r*u - 4*n*p^5*r*u - 4*L*n*p^3*r*t*u + 4*n*p^4*r*t*u

**Remark:** The parameter `a_2` does not appear in the residues !

In [27]:
for k in range(4):
    print 'Residue at {}:'.format(Sing[k])
    DM(Res[k].subs(Sols).simplify_rational().factor())
    print ''

Residue at [x == p, y == q]:


<IPython.core.display.Math object>


Residue at [x == p, y == -q]:


<IPython.core.display.Math object>


Residue at [x == t, y == r]:


<IPython.core.display.Math object>


Residue at [x == t, y == -r]:


<IPython.core.display.Math object>

In [28]:
# Position of the eight singularities:
for k in range(4):
    print Res[k].subs(Sols).simplify_rational().factor().eigenvectors_right()

[(-1/2, [(1, -2*b_3/(p - t))], 1), (1/2, [(0, 1)], 1)]
[(-1/2, [(1, -1/2*(p - t)/b_3)], 1), (1/2, [(1, 0)], 1)]
[(-1/2*n, [(1, (n*p - n*t + 2*b_3*u)/((n*p - n*t)*u + 2*b_3))], 1), (1/2*n, [(1, u)], 1)]
[(-1/2*n, [(1, ((n*p - n*t)*u + 2*b_3)/(n*p - n*t + 2*b_3*u))], 1), (1/2*n, [(1, 1/u)], 1)]


In [29]:
# Values of b_3 needed to get singularities at z = +-1

print solve((n*p - n*t + 2*b_3*u)/((n*p - n*t)*u + 2*b_3) - 1, b_3)[0]
print solve((n*p - n*t + 2*b_3*u)/((n*p - n*t)*u + 2*b_3) + 1, b_3)[0]

b_3 == 1/2*n*p - 1/2*n*t
b_3 == -1/2*n*p + 1/2*n*t


# Looking for the basis $\nabla_0, \Theta_1, \Theta_2$ used by Loray-Saito

We aim to find an explicit expression for the basis $\nabla_0, \Theta_1, \Theta_2$ used by Loray-Saito over $\mathbb{P}^1$ (after transporting these to the elliptic curve and normalizing as above).

In order to do so, first we define $\nabla_0$, $\Theta_1^+$, $\Theta_2^+$ which are characterized by a geometric property, given by the apparent map with respect to the section $z=1$.

Given a parabolic structure $(p,q,u)$, we define objects characterized by the following properties:

* $\nabla_0$ is the only connection with apparent singularities at $x=\lambda$ and $x=t$.
* $\Theta_1^+$ is the only Higgs field such that the connection $\nabla_0^+ - \Theta_1$ has apparent singularities at $w_\infty$ and $x=\lambda$.
* $\Theta_2^+$ is the only Higgs field such that the connection $\nabla_0^+ - \Theta_2$ has apparent singularities at $w_\infty$ and $x=t$.

**Remark:** This definition of the $\Theta_i^+$ differs from the $\Theta_i$ in Loray-Saito by a constant scalar factor. Indeed, using the variables $(u_1, u_2)$ introduced by LS to describe the parabolic structure over $\mathbb{P}^1$, we have that

$$ \Theta_i^+ = \frac{\eta + 1}{2(u_i - t_i)}\Theta_i . $$

Later on we'll compute $u_i$ in terms of $(p,q,u)$ and we'll be able to recover $\Theta_i$.

## Apparent map with respect to the section $\{z=1\}$

We define an apparent map giving the divisor of tangencies between $\nabla$ and the subbundle $z=1$. The divisor is of degree 4 and always contains two pairs of points in involution.

In [30]:
# Recall that M is given as follos:
DM(M)

<IPython.core.display.Math object>

Therefore we have that

$\operatorname{App}(x,y) = 2(a_2 + b_2)x^2 + 2(a_1 + b_1)x + 2(a_0 + b_0).$

Now we compute an explicit formula:

In [31]:
vect = (M.subs(Sols)).simplify_rational()*vector((1,1))
APP = (vect[0] - vect[1]).simplify_rational().numerator().factor()

### Correct values of $b_3$ and $a_2$ needed to define $\nabla_0$

We seek for the conection $\nabla_0$ which is characterized as the only connection with $\operatorname{App}(\nabla_0) \sim (x-t)(x-\lambda)$. In particular, this implies that the second singularities on the fibers over $(t,r)$ and $(t,-r)$ are on $z=1$, and the Riccati foliation has a double tangency with the line $\{z=1\}$ at $w_\lambda$.

In [32]:
# Value of b_3 needed to have the second singularity over t at z=1 is:
B3 = 1/2*(p - t)*n

In [33]:
# Verify that APP vanishes at x=t when the value of b_3 is given by B3 defined above
APP.subs(b_3 == B3).factor().subs(x==t)

0

We now find the correct value of $a_2$ such that APP vanishes at `x = L`

In [34]:
A2 = a_2.subs(solve(APP.subs([x == L, b_3 == B3]), a_2))
A2 = A2.simplify_rational().factor()
A2

1/4*(n*p^3*q*u + 2*L*n*p^2*r*u - 2*n*p^3*r*u - 2*L*n*p*q*t*u + n*p^2*q*t*u - n*p^3*q + 2*L*n*p^2*r - 2*n*p^3*r + 2*L*n*p*q*t - n*p^2*q*t + L*n*p*q*u - 2*n*p^2*q*u + 2*p^3*q*u - 2*L*n*p*r*u + 2*n*p^2*r*u + L*n*q*t*u - 2*p^2*q*t*u - L*n*p*q + 2*n*p^2*q - 2*p^3*q - 2*L*n*p*r + 2*n*p^2*r - L*n*q*t + 2*p^2*q*t - 2*p^2*q*u + 2*p*q*t*u + 2*p^2*q - 2*p*q*t)/((L - p)*(p - t)*(p - 1)*p*(u - 1))

In [35]:
# Verify that these choices of a_2, b_3 are the right ones:
APP.subs([a_2 == A2, b_3 == B3]).factor()

(L - x)*(n + 1)*(p - t)^2*(p - 1)*p*q*(t - x)*(u + 1)*(u - 1)

## Analysis of $\nabla_0$

In [36]:
nabla0 = solve(Eq + [b_3 - B3, a_2 - A2], parameters)[0]  # This is a list containing the correct value of the parameters a_i, b_i
Nabla0 = M.subs(nabla0).simplify_rational().factor()  # This is the matrix M with those correct values substituted

In [37]:
# The values of the parameters defining Nabla0
for param in parameters:
    if param.subs(nabla0) == 0:
        print '{} = 0'.format(param), '\n'
    else:
        print '{} ='.format(param), param.subs(nabla0).simplify_rational().factor(), '\n'

a_0 = 1/4*(2*L*n*p^3*r*u - 2*n*p^4*r*u + n*p^3*q*t*u - 2*L*n*p*q*t^2*u + n*p^2*q*t^2*u + 2*L*n*p^3*r - 2*n*p^4*r - n*p^3*q*t + 2*L*n*p*q*t^2 - n*p^2*q*t^2 - 2*L*n*p^2*r*u + 2*n*p^3*r*u + L*n*p*q*t*u + 2*L*p^2*q*t*u - 2*n*p^2*q*t*u + L*n*q*t^2*u - 2*L*p*q*t^2*u - 2*L*n*p^2*r + 2*n*p^3*r - L*n*p*q*t - 2*L*p^2*q*t + 2*n*p^2*q*t - L*n*q*t^2 + 2*L*p*q*t^2 - 2*L*p*q*t*u + 2*L*q*t^2*u + 2*L*p*q*t - 2*L*q*t^2)/((L - p)*(p - t)*(p - 1)*(u - 1)) 

a_1 = -1/4*(n*p^4*q*u + 4*L*n*p^3*r*u - 4*n*p^4*r*u - 2*L*n*p^2*q*t*u + 2*n*p^3*q*t*u - 2*L*n*p*q*t^2*u + n*p^2*q*t^2*u - n*p^4*q + 4*L*n*p^3*r - 4*n*p^4*r + 2*L*n*p^2*q*t - 2*n*p^3*q*t + 2*L*n*p*q*t^2 - n*p^2*q*t^2 + L*n*p^2*q*u + 2*L*p^3*q*u - 2*n*p^3*q*u - 4*L*n*p^2*r*u + 4*n*p^3*r*u + 2*L*n*p*q*t*u - 2*L*p^2*q*t*u - 2*n*p^2*q*t*u + 2*p^3*q*t*u + L*n*q*t^2*u - 2*p^2*q*t^2*u - L*n*p^2*q - 2*L*p^3*q + 2*n*p^3*q - 4*L*n*p^2*r + 4*n*p^3*r - 2*L*n*p*q*t + 2*L*p^2*q*t + 2*n*p^2*q*t - 2*p^3*q*t - L*n*q*t^2 + 2*p^2*q*t^2 - 2*L*p^2*q*u + 2*L*p*q*t*u - 2*p^2*

In [38]:
# Residues of Nabla0
for k in range(4):
    print 'Residue at {}:'.format(Sing[k])
    residue = Res[k].subs(nabla0).simplify_rational().factor()
    DM(residue)
    print 'Eigenvectors:', residue.eigenvectors_right()
    print '\n'

Residue at [x == p, y == q]:


<IPython.core.display.Math object>

Eigenvectors: [(-1/2, [(1, -n)], 1), (1/2, [(0, 1)], 1)]


Residue at [x == p, y == -q]:


<IPython.core.display.Math object>

Eigenvectors: [(-1/2, [(1, -1/n)], 1), (1/2, [(1, 0)], 1)]


Residue at [x == t, y == r]:


<IPython.core.display.Math object>

Eigenvectors: [(-1/2*n, [(1, 1)], 1), (1/2*n, [(1, u)], 1)]


Residue at [x == t, y == -r]:


<IPython.core.display.Math object>

Eigenvectors: [(-1/2*n, [(1, 1)], 1), (1/2*n, [(1, 1/u)], 1)]




## Apparent map once more

I will first define naive candidates for $\Theta_1$ and $\Theta_2$ and then, via the apparent map, figure out which linear combinations of these are the right ones. 

In [39]:
to_zero = [a_0==0, a_1==0, a_2==0, b_0==0, b_1==0, b_2==0, b_3==0]
EqHom = [eq - eq.subs(to_zero) for eq in Eq]

In [40]:
theta1_naive = solve(EqHom + [b_3 - 1, a_2], parameters)[0]  # set b3=1, a2=0
theta2_naive = solve(EqHom + [b_3, a_2 - 1], parameters)[0]  # set b3=0, a2=1

In [41]:
Theta1_n = M.subs(theta1_naive).simplify_rational().factor()
Theta2_n = M.subs(theta2_naive).simplify_rational().factor()

In [42]:
Mbis = (Nabla0 + c1*Theta1_n + c2*Theta2_n).simplify_rational().factor()

In [43]:
vect = Mbis*vector((1,1))
APP_naive = (vect[0] - vect[1]).simplify_rational().numerator().factor()

In [44]:
# Check: this should return c1=0, c2=0
solve([APP_naive.subs(x==t), APP_naive.subs(x==L)], [c1, c2])

[[c1 == 0, c2 == 0]]

In [45]:
# Theta2 is defined by having apparent singularities at infinity and t
SolsTheta2 = solve([diff(diff(APP_naive, x), x), APP_naive.subs(x==t)], [c1, c2])[0]
print SolsTheta2

[c1 == 0, c2 == -1/2*(n + 1)*q/(L - p)]


In [46]:
# Theta1 is defined by having apparent singularities at infinity and L
SolsTheta1 = solve([diff(diff(APP_naive, x), x), APP_naive.subs(x==L)], [c1, c2])[0]

print 'c1 =', c1.subs(SolsTheta1).simplify_rational().factor(), '\n'
print 'c2 =', c2.subs(SolsTheta1).simplify_rational().factor(), '\n'

c1 = -1/2*(L - t)*(n + 1)*(p - t)*q*(u + 1)/(L*q*u - L*r*u + p*r*u - q*t*u + L*q + L*r - p*r - q*t) 

c2 = -1/4*(L*p^3*q*u^2 + 2*L^2*p^2*r*u^2 - 4*L*p^3*r*u^2 + 2*p^4*r*u^2 - 2*L^2*p*q*t*u^2 + L*p^2*q*t*u^2 - p^3*q*t*u^2 + 2*L*p*q*t^2*u^2 - p^2*q*t^2*u^2 + 4*L*p^3*r*u - 4*p^4*r*u - 4*L*p^2*r*t*u + 4*p^3*r*t*u + L^2*p*q*u^2 - 2*L*p^2*q*u^2 - 2*L^2*p*r*u^2 + 4*L*p^2*r*u^2 - 2*p^3*r*u^2 + L^2*q*t*u^2 - L*p*q*t*u^2 + 2*p^2*q*t*u^2 - L*q*t^2*u^2 - L*p^3*q + 2*L^2*p^2*r - 4*L*p^3*r + 2*p^4*r + 2*L^2*p*q*t - L*p^2*q*t + p^3*q*t - 2*L*p*q*t^2 + p^2*q*t^2 - 4*L*p^2*r*u + 4*p^3*r*u + 4*L*p*r*t*u - 4*p^2*r*t*u - L^2*p*q + 2*L*p^2*q - 2*L^2*p*r + 4*L*p^2*r - 2*p^3*r - L^2*q*t + L*p*q*t - 2*p^2*q*t + L*q*t^2)*(n + 1)*q/((L*q*u - L*r*u + p*r*u - q*t*u + L*q + L*r - p*r - q*t)*(L - p)*(p - t)*(p - 1)*p*(u - 1)) 



# Definition of $\Theta_1^+$ and $\Theta_2^+$

In [47]:
theta1plus = solve(EqHom + [b_3 + c1.subs(SolsTheta1), a_2 + c2.subs(SolsTheta1)], parameters)[0]
Theta1plus = M.subs(theta1plus).simplify_rational().factor()

In [48]:
theta2plus = solve(EqHom + [b_3, a_2 + c2.subs(SolsTheta2)], parameters)[0]
Theta2plus = M.subs(theta2plus).simplify_rational().factor()

**Remark:** The explicit expression of $\Theta_2^+$ is given by:

$$ \Theta_2^+ = \frac{q(\eta+1)}{2(\lambda-p)}\pmatrix{1 & 0\\ 0 & -1}\frac{dx}{y}.$$

## Final version of the Apparent Map using the basis $\nabla_0, \Theta_1^+, \Theta_2^+$

In [49]:
c0 = var('c0')

In [50]:
Nabla = c0*Nabla0 + c1*Theta1plus + c2*Theta2plus
Nabla = Nabla.simplify_rational().factor()

In [51]:
vect = Nabla*vector((1,1))
APP = (vect[0] - vect[1]).simplify_rational().numerator().factor()

In [52]:
print APP

-(L^2*c1*p*r*u - L*c1*p^2*r*u - L^2*c0*q*t*u - L^2*c1*q*t*u - L*c2*p*q*t*u + L^2*c0*r*t*u - L*c0*p*r*t*u + L*c2*p*r*t*u - c2*p^2*r*t*u + L*c0*q*t^2*u + L*c1*q*t^2*u + c2*p*q*t^2*u + L^2*c0*q*u*x + L^2*c1*q*u*x + L*c2*p*q*u*x - L^2*c0*r*u*x - L^2*c1*r*u*x + L*c0*p*r*u*x - L*c2*p*r*u*x + c1*p^2*r*u*x + c2*p^2*r*u*x + L*c2*q*t*u*x - c2*p*q*t*u*x - L*c0*r*t*u*x - L*c2*r*t*u*x + c0*p*r*t*u*x + c2*p*r*t*u*x - c0*q*t^2*u*x - c1*q*t^2*u*x - c2*q*t^2*u*x - L*c0*q*u*x^2 - L*c1*q*u*x^2 - L*c2*q*u*x^2 + L*c0*r*u*x^2 + L*c1*r*u*x^2 + L*c2*r*u*x^2 - c0*p*r*u*x^2 - c1*p*r*u*x^2 - c2*p*r*u*x^2 + c0*q*t*u*x^2 + c1*q*t*u*x^2 + c2*q*t*u*x^2 - L^2*c1*p*r + L*c1*p^2*r - L^2*c0*q*t - L^2*c1*q*t - L*c2*p*q*t - L^2*c0*r*t + L*c0*p*r*t - L*c2*p*r*t + c2*p^2*r*t + L*c0*q*t^2 + L*c1*q*t^2 + c2*p*q*t^2 + L^2*c0*q*x + L^2*c1*q*x + L*c2*p*q*x + L^2*c0*r*x + L^2*c1*r*x - L*c0*p*r*x + L*c2*p*r*x - c1*p^2*r*x - c2*p^2*r*x + L*c2*q*t*x - c2*p*q*t*x + L*c0*r*t*x + L*c2*r*t*x - c0*p*r*t*x - c2*p*r*t*x - c0*q*t^2*x - c1*q

The above polynomial can be written as:

$$ c_0A_0(x-t)(x-\lambda) + c_1A_1(x-\lambda) + c_2A_2(x-t),$$

with $A_0$ independent of $x$, and $A_1,A_2$ linear polynomials on $x$. 

We are interested in the roots of $A_1,A_2$ (the "apparent singularities" of $\Theta_i$). In Loray-Saito these are called $\mu_i$ and are given by the formula:

$$ \mu_i = \frac{t_i(u_i-1)}{u_i - t_i} .$$

In fact, we shall compute $\mu_i$ and from these we will be able to recover $u_i$ in terms of our variables $(p,q,u)$.

In [53]:
APP0 = diff(APP, c0).simplify_rational().factor()
APP1 = diff(APP, c1).simplify_rational().factor()
APP2 = diff(APP, c2).simplify_rational().factor()

In [54]:
# Note that mu2 = p:
solve((APP2/(x-t)).simplify_rational(), x)

[x == p]

In [133]:
# Definition of mu1
sol_mu1 = solve((APP1/(x-L)).simplify_rational(), x)[0]

mu1 = x.subs(sol_mu1).simplify_rational().factor()
DM(mu1)

<IPython.core.display.Math object>

Using the basis $\nabla_0, \Theta_1^+, \Theta_2^+$, the apparent map is given by:

$$\operatorname{App}(\nabla) = (x-t)(x-\lambda) + c_1(x-\lambda)(x-\mu_1) + c_2(x-t)(x-\mu_2),$$

where $\mu_1$ is computed above and $\mu_2 = p$.

## The map $(u,p,q) \mapsto (u_1,u_2)$

In [56]:
var('u1 u2')
t1 = t
t2 = L

In [57]:
# Use mu1, mu2 to solve for u1, u2
Sols_u1u2 = solve([t1*(u1-1)/(u1-t1) == mu1, t2*(u2-1)/(u2-t2) == p], [u1,u2])[0]

In [58]:
print 'u1 =', u1.subs(Sols_u1u2).simplify_rational().factor(), '\n'
print 'u2 =', u2.subs(Sols_u1u2).simplify_rational().factor()

u1 = (L*p*r*u - p^2*r*u - L*q*t*u + q*t^2*u - L*p*r + p^2*r - L*q*t + q*t^2 + L*q*u - L*r*u + p*r*u - q*t*u + L*q + L*r - p*r - q*t)*t/((L - p)*(p - t)*r*(u - 1)) 

u2 = -L*(p - 1)/(L - p)


We can do the following to avoid having `r` in the denominator of `u1`

In [59]:
print 'u1 =', simplify_qr(u1.subs(Sols_u1u2)*r^2)/r2 

u1 = (L*p*t*u - p^2*t*u - L*p*t + p^2*t + q*r*u - L*t*u + p*t*u + q*r + L*t - p*t)/((L - p)*(p - t)*(u - 1))


## The map $(u_1,u_2) \mapsto (p,q,u)$

In [60]:
# We need to introduce the following variable
v2 = var('v2')  # v2 = sqrt(u2*(u2-1)*(u2-L))

In [62]:
# Use mu1, mu2 to solve for p,q,u
Sols_pqu = solve([t1*(u1-1)/(u1-t1) == mu1.subs(q == (L^2 - L)*v2/(L^2 - 2*L*u2 + u2^2)), t2*(u2-1)/(u2-t2) == p], [u,p])[0]

In [63]:
# Write q in terms of v2

# The quantity below is equal to one:
expr = (q^2/(p*(p-1)*(p-L))) / (v2^2/(u2*(u2-1)*(u2-L)))

# Substitute p for its formula in terms of u2
expr = expr.subs(Sols_pqu)

# Solve for q
solve(expr == 1, q)  # Let's choose the second root

[q == -(L^2 - L)*v2/(L^2 - 2*L*u2 + u2^2), q == (L^2 - L)*v2/(L^2 - 2*L*u2 + u2^2)]

In [64]:
print 'u =', u.subs(Sols_pqu).simplify_rational().factor(), '\n'
print 'p =', p.subs(Sols_pqu).simplify_rational().factor(), '\n'
print 'q =', q.subs(q == (L^2 - L)*v2/(L^2 - 2*L*u2 + u2^2)).factor()

u = (L*r*t*u1 - L*r*t*u2 + L*r*u1*u2 - r*t*u1*u2 + L*t^2*v2 - t^3*v2 - L*r*u1 + r*t*u2 - L*t*v2 + t^2*v2)/(L*r*t*u1 - L*r*t*u2 + L*r*u1*u2 - r*t*u1*u2 - L*t^2*v2 + t^3*v2 - L*r*u1 + r*t*u2 + L*t*v2 - t^2*v2) 

p = -L*(u2 - 1)/(L - u2) 

q = (L - 1)*L*v2/(L - u2)^2


## Recovery of the original Higgs fields $\Theta_i$ in Loray-Saito

Now that we have computed the value of $u_1$ in terms of our variables we can rescale the $\Theta_i^+$ back to $\Theta_i$.

In [65]:
rescale_factor1 = ((n + 1)/(2*(u1 - t1))).subs(Sols_u1u2).simplify_rational().factor()
rescale_factor2 = ((n + 1)/(2*(u2 - t2))).subs(Sols_u1u2).simplify_rational().factor()

In [66]:
Theta1 = (Theta1plus/rescale_factor1).simplify_rational().factor()

Theta2 = (Theta2plus/rescale_factor2).simplify_rational().factor()

# First automorphism of the familiy: Permuting the roles of $p_1 = (p,q)$ and $p_2=(p,-q)$

Permuting the numbering of $p_1,p_2$ and then applying $z\mapsto z^{-1}$ transforms a connection into an isomorphic one. In our family, this corresponds to the transformation:

$M(u,p,q) \longmapsto  \mathrm{P}\,M(u^{-1}, p, -q)\,\mathrm{P}^{-1} $,

where $\mathrm{P}$ is the permutation matrix that interchanges the two colums of $M$.

**Remark:** The objects $\nabla_0$, and $\Theta_i^+$ constructed above are invariant under this transformation.

In [70]:
PM = matrix([(0,1), (1,0)])

In [71]:
# Nabla0plus is invariant
(Nabla0 - PM*Nabla0.subs([q==-q, u==1/u])*PM).simplify_rational().factor() == 0

True

In [72]:
# Theta1plus is invariant
(Theta1plus - PM*Theta1plus.subs([q==-q, u==1/u])*PM).simplify_rational().factor() == 0

True

In [73]:
# Theta2 is invariant
(Theta2plus - PM*Theta2plus.subs([q==-q, u==1/u])*PM).simplify_rational().factor() == 0

True

*Remark:* Also the expression for $\mu_1$ is invariant under the substitution $(u,p,q)\mapsto(1/u,p,-q)$.

In [74]:
(mu1 - mu1.subs([q==-q, u==1/u])).simplify_rational()

0

We don't need to define + and - versions of $\Theta_2$, since the first expression is already invariant.

# Second automorphism of the familiy: The involution $\tau\;\colon z \mapsto -z$

In our trivial bundle $\mathbb{P}^1_z \times C$ we have the bundle automorphism $\tau\colon z\mapsto -z$. A connection $\nabla$ is transformed to $\tau^*\nabla = \Lambda\nabla\Lambda$, where $\Lambda$ is the diagonal matrix $\operatorname{diag}(1,-1)$.

If the parabolic structure of $\nabla$ was given by $(p,q,u)$, then the parabolic structure of $\tau^*\nabla$ will be $(p,q,-u)$.

Our basic elements $\nabla_0$ and $\Theta_1$ are not equivariant with respect to $\tau$:
$$ \tau^*\nabla_0(p,q,u) \neq \nabla_0(p,q,-u), \quad \tau^*\Theta_1(p,q,u) \neq \Theta_1(p,q,-u) . $$

We will now introduce a new basis that is equivariant with respect to $\tau$.

$$ \hat\nabla_0(p,q,u) = \frac{1}{2}\left( \nabla_0(p,q,u) + \tau^*\nabla_0(p,q,-u) \right), \quad \hat\Theta_1(p,q,u) = \frac{1}{2}\left( \Theta_1(p,q,u) + \tau^*\Theta_1(p,q,-u) \right), \quad  \hat\Theta_2(p,q,u) = \Theta_2(p,q,u) . $$

In [75]:
Lambda = diagonal_matrix([1, -1])

In [76]:
Nabla0_hat = (1/2)*(Nabla0 + Lambda*Nabla0.subs(u==-u)*Lambda).simplify_rational().factor()

In [77]:
Theta1_hat = (1/2)*(Theta1 + Lambda*Theta1.subs(u==-u)*Lambda).simplify_rational().factor()

In [78]:
Theta2_hat = Theta1

Note that this basis has the following property:

If $\nabla = \hat\nabla_0(p,q,u) + k_1\hat\Theta_1(p,q,u) + k_2\hat\Theta_2(p,q,u)$,

then $\tau^*\nabla = \hat\nabla_0(p,q,-u) + k_1\hat\Theta_1(p,q,-u) + k_2\hat\Theta_2(p,q,-u)$.

This means that $\tau$ acts on our family of connections as $(p,q,u,k_1,k_2) \mapsto (p,q,-u,k_1,k_2)$. We will use this property to push our family and the symplectic 2-form onto the moduli space of connections.

## The change of basis $(\nabla_0, \Theta_1, \Theta_2) \mapsto (\hat\nabla_0, \hat\Theta_1, \hat\Theta_2)$

We seek for a matrix T with the following property:

$$ \text{If }\; \nabla = \nabla_0(p,q,u) + c_1\Theta_1(p,q,u) + c_2\Theta_2(p,q,u), \;\text{ then }\; \tau^*\nabla = \nabla_0(p,q,-u) + k_1\Theta_1(p,q,-u) + k_2\Theta_2(p,q,-u), $$

where $(1, k_1, k_2)$ is the image of $(1, c_1, c_2)$ under the matrix $T$.

In order to find $T$ we are going to look at the tangencies of a connection with the section $z = -1$, and use the fact that the points of tangency of $\nabla$ with $z = 1$ are exactly the points of tangency of $\tau^*\nabla$ with $z = -1$.

In [85]:
var('T10 T11 T20 T21')
T = matrix([(1, 0 ,0), (T10, T11, 0), (T20, T21, 1)])
T

[  1   0   0]
[T10 T11   0]
[T20 T21   1]

In [89]:
k0, k1, k2 = list(T*vector([1, c1, c2]))

In [104]:
Nabla = Nabla0 + c1*Theta1plus/rescale_factor1 + c2*Theta2plus/rescale_factor2
Nabla = Nabla.simplify_rational().factor()

Tangencies of $\nabla = \nabla_0(p,q,u) + c_1\Theta_1(p,q,u) + c_2\Theta_2(p,q,u)$ with $z = 1$:

In [106]:
vect = Nabla*vector((1,1))
APPplus = (vect[0] - vect[1]).simplify_rational().numerator()

Tangencies of $\tau^*\nabla = \nabla_0(p,q,-u) + k_1\Theta_1(p,q,-u) + k_2\Theta_2(p,q,-u)$ with $z = -1$:

In [107]:
vect = Nabla*vector((1,-1))
APPminus = (vect[0] + vect[1])

conjugate_APP = APPminus.subs(u==-u).subs([c1==k1, c2==k2]).simplify_rational().numerator()  # This should have the same zeros as APPplus of nabla

### Comparison of the two polynomials

In [108]:
ap0 = APPplus.subs(x==0)
ap1 = diff(APPplus, x).subs(x==0)
ap2 = diff(diff(APPplus/2, x), x)

In [109]:
ac0 = conjugate_APP.subs(x==0)
ac1 = diff(conjugate_APP, x).subs(x==0)
ac2 = diff(diff(conjugate_APP/2, x), x)

In [110]:
Eqn1 = ac2*ap1 - ap2*ac1
Eqn2 = ac2*ap0 - ap2*ac0

I want the above equations to be zero **for any** value of $c_1,c_2$.  
The expressions `Eqn1` and `Eqn2` are quadratic in $c_1,c_2$, but we'll truncate the quadratic part first, solve the system and then show that the solutions found are general enough.

In [111]:
# Linear terms

E1_00 = Eqn1.subs([c1==0, c2==0])
E1_10 = diff(Eqn1, c1).subs([c1==0, c2==0])
E2_00 = Eqn2.subs([c1==0, c2==0])
E2_10 = diff(Eqn2, c1).subs([c1==0, c2==0])

In [112]:
%%time
# This might take a while

# Solve by column of T
S0 = solve([E1_00, E2_00], [T10, T20])
S1 = solve([E1_10.subs(S0), E2_10.subs(S0)], [T11, T21])

# Verify that we got a complete solution
print Eqn1.subs(S0).subs(S1).simplify_rational()
print Eqn2.subs(S0).subs(S1).simplify_rational()

0
0
CPU times: user 1min 51s, sys: 252 ms, total: 1min 51s
Wall time: 1min 45s


### Final expression for the Conjugacy Matrix `CM`

In [113]:
v0 = (1, simplify_qr(T10.subs(S0)*q^2/q2), simplify_qr(T20.subs(S0)*q^2/q2))
v1 = (0, T11.subs(S1).factor(), simplify_qr(T21.subs(S1)*r^2/r2))
v2 = (0, 0, 1)

CM = matrix([v0, v1, v2]).transpose()

In [115]:
# Let's verify that CM actually defines an involution
DM((CM * CM.subs(u==-u)).simplify_rational().factor())

<IPython.core.display.Math object>

In [116]:
# The expression in the corner actually simplifies to zero:
simplify_qr((CM * CM.subs(u==-u))[2][0])

0

## The Base Change matrix `BC`

In [117]:
BC = (identity_matrix(3) + CM.subs(u==-u))/2
BC = BC.simplify_rational().factor()

In [118]:
# Print row by row to export:
for k in range(3):
    print 'row{} ='.format(k), BC[k], '\n'

row0 = (1, 0, 0) 

row1 = (-1/2*n*(p - t)*q*r*(u - 1)/((L - t)*(p - 1)*p*(t - 1)*t*(u + 1)), 2*u/(u + 1)^2, 0) 

row2 = (-1/4*(L*p^3*u - 2*L^2*p*t*u + L*p^2*t*u - p^3*t*u + 2*L*p*t^2*u - p^2*t^2*u + L*p^3 - 2*L^2*p*t + L*p^2*t - p^3*t + 2*L*p*t^2 - p^2*t^2 + L^2*p*u - 2*L*p^2*u - 2*L*q*r*u + 2*p*q*r*u + L^2*t*u - L*p*t*u + 2*p^2*t*u - L*t^2*u + L^2*p - 2*L*p^2 + 2*L*q*r - 2*p*q*r + L^2*t - L*p*t + 2*p^2*t - L*t^2)*(L - p)*n/((L - t)*(L - 1)*L*(p - t)*(p - 1)*p*(u + 1)), 1/2*(2*L^2*p^2*t^2*u^2 - 4*L*p^3*t^2*u^2 + 2*p^4*t^2*u^2 - p^3*q*r*u^2 - 2*L^2*p^2*t*u^2 + 4*L*p^3*t*u^2 - 2*p^4*t*u^2 + 2*L*p*q*r*t*u^2 - p^2*q*r*t*u^2 - 2*L^2*p*t^2*u^2 + 4*L*p^2*t^2*u^2 - 2*p^3*t^2*u^2 + 2*L^2*p^2*t^2 - 4*L*p^3*t^2 + 2*p^4*t^2 - L*p*q*r*u^2 + 2*p^2*q*r*u^2 + 2*L^2*p*t*u^2 - 4*L*p^2*t*u^2 + 2*p^3*t*u^2 - L*q*r*t*u^2 + p^3*q*r - 2*L^2*p^2*t + 4*L*p^3*t - 2*p^4*t - 2*L*p*q*r*t + p^2*q*r*t - 2*L^2*p*t^2 + 4*L*p^2*t^2 - 2*p^3*t^2 + L*p*q*r - 2*p^2*q*r + 2*L^2*p*t - 4*L*p^2*t + 2*p^3*t + L*q*r*t)/((L - 1)*

# The map into the moduli space of parabolic bundles $\mathbb{P}^1_z\times\mathbb{P}^1_w$.

Suppose we start with a parabolic bundle on $\mathbb{P}^1$ having parabolic structure $(0,1,c,\infty,l)$ over the points $0,1,\lambda,\infty,t$ respectively. After pulling back to the elliptic curve and mapping this parabolic bundle to its moduli space, we recover a point $(z,w) \in \mathbb{P}^1_z \times \mathbb{P}^1_w$. This point was explicitly computed by Nestor, and the values for $z,w$ are the following:

In [121]:
var('w')
dict_vars['w'] = w

In [122]:
# Nestor's formula CORRECTED:
zw_vals = [
    z == t2*(u2-1)/(u2-t2),
    w == (t2*u1*(t2*(u1-1) - t1*(u2-1) + u2 - u1))/(t2*(t1*(u1-u2) + u1*(u2-1)) - u2*t1*(u1-1))
]

Using the variables $\mu_i$ (which represent the apparent singularities of $\Theta_i$), we can rewrite the formulas for $z,w$ as follows:

$$ z = \mu_2, \quad w = \frac{(t_2\mu_1 - t_1\mu_2)(\mu_1 - 1)}{(\mu_1 - \mu_2)(\mu_1 - t_1)} . $$

In [132]:
# Verify the above claim:
var('Mu1 Mu2')
to_mu = [u1==t1*(Mu1-1)/(Mu1-t1), u2==t2*(Mu2-1)/(Mu2-t2)]

print 'z =', z.subs(zw_vals).subs(to_mu).simplify_rational().factor()
print 'w =', w.subs(zw_vals).subs(to_mu).simplify_rational().factor()

z = Mu2
w = (L*Mu1 - Mu2*t)*(Mu1 - 1)/((Mu1 - Mu2)*(Mu1 - t))


In [135]:
formula_w = w.subs(zw_vals).subs(to_mu).subs([Mu1==mu1, Mu2==p]).simplify_rational().factor()
print formula_w

(L*p*r*u - p^2*r*u - L*q*t*u + q*t^2*u - L*p*r + p^2*r - L*q*t + q*t^2 + L*q*u - L*r*u + p*r*u - q*t*u + L*q + L*r - p*r - q*t)*(p*r*u - q*t*u - p*r - q*t)/((p - t)^2*q*r*(u + 1)*(u - 1))


Simplify the expression for `w` to avoid having `q` or `r` in the denominator

In [136]:
formula_w = simplify_qr(formula_w*r^2*q^2/q2/r2)

In [137]:
DM(formula_w)

<IPython.core.display.Math object>

The above expression is invariant under all automorphisms of our family (as expected):

In [143]:
(formula_w - formula_w.subs([u==1/u, q==-q])).simplify_rational()

0

In [144]:
(formula_w - formula_w.subs(u==-u)).simplify_rational()

0

In [145]:
(formula_w - formula_w.subs([u==1/u, r==-r])).simplify_rational()

0

# Computations for the pullback of the symplectic 2-form $\omega$

Let $\Psi$ be the map from the family of connections on the elliptic curve onto the family of connections on $\mathbb{P}^1$.

Over $C$ we use the basis $\hat\nabla_0, \hat\Theta_1, \hat\Theta_2$ which is equivariant with respect to the involution $\tau$. Using this basis, any connection is defined by parameters $p,q,u,\kappa_1,\kappa_2$. On $\mathbb{P}^1 we use the basis introduced by Loray-Saito. Thus, our map looks like:

$$ \Psi \colon (p,q,u,\kappa_1,\kappa_2) \longmapsto (u_1, u_2, c_1, c_2), $$

where the transformation on the $c_i$

$$ (1, \kappa_1, \kappa_2) \longmapsto (1, c_1, c_2), $$

is given by the "base change matrix" `BC = (1/2)*(Identity + CM)`, where `CM` is the action of $\tau$ on the coefficients $c_i$.

In [149]:
# Reset the variables k1,k2 and c1,c2
del k1, k2, c1, c2

k1, k2 = var('k1 k2')
dict_vars['k1'] = k1
dict_vars['k2'] = k2

## Set the variables $(c_1, c_2) = \operatorname{BC}(\kappa_1, \kappa_2)$

In [152]:
c1 = (BC*vector([1, k1, k2]))[1]
c2 = (BC*vector([1, k1, k2]))[2]

## Computations for $\omega_C = \Psi^*(dc_1\wedge du_1 + dc_2\wedge du_2)$

We'll store all differentials as dictionaries. For example, `dc1` will be a dictionary with keys: 'dk1', 'dk2', 'du', 'dp'; in such a way that `dc1['dk1']` gives the `dk1` coefficient in the differential `dc1`.

In [154]:
# Differential of q
dq = dP.subs(x==p)*q/(2*P.subs(x==p))  # ie P'(p)/2q * q^2/P(p)

In [155]:
# Compute dc1
variable = c1

num = variable.numerator()
den = variable.denominator()

# Derivative with respect to p

assert diff(diff(num, q), q) == 0
assert diff(den, q) == 0

# Split into q-part and q-free-part
num0 = num.subs(q==0)
num1 = diff(num, q)

# Compute derivatives wrt p
dnum_dp = diff(num0, p) + q*diff(num1, p) + dq*num1
dden_dp = diff(den, p)
dvar_dp = (dnum_dp*den - num*dden_dp)/den^2
dvar_dp = dvar_dp.simplify_rational().factor()

# Save differential as a dictionary
dc1 = {}
dc1['dk1'] = diff(variable, k1).simplify_rational().factor()
dc1['dk2'] = diff(variable, k2)  # this one is zero
dc1['du'] = diff(variable, u).simplify_rational().factor()
dc1['dp'] = dvar_dp

In [156]:
# Compute dc2
variable = c2
variable = simplify_qr(variable)  # get rid of higher powers of q

num = variable.numerator()
den = variable.denominator()

# Derivative with respect to p

assert diff(diff(num, q), q) == 0
assert diff(den, q) == 0

# Split into q-part and q-free-part
num0 = num.subs(q==0)
num1 = diff(num, q)

# Compute derivatives wrt p
dnum_dp = diff(num0, p) + q*diff(num1, p) + dq*num1
dden_dp = diff(den, p)
dvar_dp = (dnum_dp*den - num*dden_dp)/den^2
dvar_dp = dvar_dp.simplify_rational().factor()

# Save differential as a dictionary
dc2 = {}
dc2['dk1'] = diff(variable, k1).simplify_rational().factor()
dc2['dk2'] = diff(variable, k2).simplify_rational().factor()
dc2['du'] = diff(variable, u).simplify_rational().factor()
dc2['dp'] = dvar_dp

In [159]:
# Compute du1
variable = u1.subs(Sols_u1u2)  # formula for u1 found above
variable = simplify_qr(variable*r^2/r2)  # get rid of r in the denominator

num = variable.numerator()
den = variable.denominator()

# Derivative with respect to p

assert diff(diff(num, q), q) == 0
assert diff(den, q) == 0

# Split into q-part and q-free-part
num0 = num.subs(q==0)
num1 = diff(num, q)

# Compute derivatives wrt p
dnum_dp = diff(num0, p) + q*diff(num1, p) + dq*num1
dden_dp = diff(den, p)
dvar_dp = (dnum_dp*den - num*dden_dp)/den^2
dvar_dp = dvar_dp.simplify_rational().factor()

# Save differential as a dictionary
du1 = {}
du1['dk1'] = 0
du1['dk2'] = 0
du1['du'] = diff(variable, u).simplify_rational().factor()
du1['dp'] = dvar_dp

In [160]:
# Compute du2
variable = u2.subs(Sols_u1u2)  # formula for u2 found above

# Save differential as a dictionary
du2 = {}
du2['dk1'] = 0
du2['dk2'] = 0
du2['du'] = 0
du2['dp'] = diff(variable, p).simplify_rational().factor()

**Putting it all together**

In [162]:
keys = ['dk1^du', 'dk1^dp', 'dk2^du', 'dk2^dp', 'dk1^dk2', 'du^dp']

In [163]:
# Initiallize omega to zeros
omega = {key: 0 for key in keys}

# Coming from dc1^du1
omega['dk1^du'] += dc1['dk1'] * du1['du']
omega['dk1^dp'] += dc1['dk1'] * du1['dp']
omega['dk2^du'] += dc1['dk2'] * du1['du']
omega['dk2^dp'] += dc1['dk2'] * du1['dp']
omega['du^dp']  += dc1['du'] * du1['dp']
omega['du^dp']  += - dc1['dp'] * du1['du']

# Coming from dc2^du2
omega['dk1^dp'] += dc2['dk1'] * du2['dp']
omega['dk2^dp'] += dc2['dk2'] * du2['dp']
omega['du^dp']  += dc2['du'] * du2['dp']

In [164]:
# Simplify the above terms:
for key in keys:
    try:
        omega[key] = omega[key].simplify_rational()
    except:
        next
    try:
        omega[key] = omega[key].factor()
    except:
        next

In [167]:
omega['du^dp']

0

In [169]:
# Simplify more
omega['dk1^dp'] = simplify_qr(omega['dk1^dp']*r^2/r2)
omega['du^dp'] = simplify_qr(omega['du^dp'])

In [170]:
# Print omega term by term
for key in keys:
    print '{} term:'.format(key)
    print(omega[key])
    print ''

dk1^du term:
-4*q*r*u/((L - p)*(p - t)*(u + 1)^2*(u - 1)^2)

dk1^dp term:
-1/2*(2*L^2*p^2*t^2*u^2 - 4*L*p^3*t^2*u^2 + 2*p^4*t^2*u^2 - p^3*q*r*u^2 - 2*L^2*p^2*t*u^2 + 4*L*p^3*t*u^2 - 2*p^4*t*u^2 + 2*L*p*q*r*t*u^2 - p^2*q*r*t*u^2 - 2*L^2*p*t^2*u^2 + 4*L*p^2*t^2*u^2 - 2*p^3*t^2*u^2 - 2*L^2*p^2*t^2 + 4*L*p^3*t^2 - 2*p^4*t^2 - L*p*q*r*u^2 + 2*p^2*q*r*u^2 + 2*L^2*p*t*u^2 - 4*L*p^2*t*u^2 + 2*p^3*t*u^2 - L*q*r*t*u^2 - p^3*q*r + 2*L^2*p^2*t - 4*L*p^3*t + 2*p^4*t + 2*L*p*q*r*t - p^2*q*r*t + 2*L^2*p*t^2 - 4*L*p^2*t^2 + 2*p^3*t^2 - L*p*q*r + 2*p^2*q*r - 2*L^2*p*t + 4*L*p^2*t - 2*p^3*t - L*q*r*t)/((L - p)^2*(p - t)^2*(p - 1)*p*(u + 1)*(u - 1))

dk2^du term:
0

dk2^dp term:
-(L - 1)*L/(L - p)^2

dk1^dk2 term:
0

du^dp term:
0



Invariance checks:

In [171]:
print (omega['dk1^du'] - (-1)*omega['dk1^du'].subs(u==-u)).simplify_rational()
print (omega['dk1^dp'] - omega['dk1^dp'].subs(u==-u)).simplify_rational()
print (omega['dk2^dp'] - omega['dk2^dp'].subs(u==-u)).simplify_rational()

0
0
0


In [172]:
print (omega['dk1^du'] - (-1/u^2)*omega['dk1^du'].subs([q==-q, u==1/u])).simplify_rational()
print (omega['dk1^dp'] - omega['dk1^dp'].subs([q==-q, u==1/u])).simplify_rational()
print (omega['dk2^dp'] - omega['dk2^dp'].subs([q==-q, u==1/u])).simplify_rational()

0
0
0


# Rewrite everything in terms of the variable $\mu = qr(u^2 + 1)/(u^2 - 1)$

The expression $\mu$ above is the simplest function that is invariant under all transformations:

* `[q == -q, u == 1/u]`, corresponding to permuting the labels of the points $(p,q)$ and $(p,-q)$ and applying $z \mapsto 1/z$.

* `[u == -u]`, corresponding to the involution $\tau: z \mapsto -z$.

* `[r == -r, u == 1/u]`, corresponding to permuting the labels of the points $(t,r)$ and $(t,-r)$.

**Strategy:** If we use this variable $\mu$ we should be able to express everything in terms of $p$ and $\mu$ only, thus avoiding the "square roots" $q$ and $r$.

Note that we have the following identity: 

$$ d\mu = \frac{d\mu}{u}du + \mu \frac{dq}{q}. $$

Then, unwinding all the expressions we get:

$$d\mu = -\frac{4qru}{(u^2-1)^2}du + \mu \frac{P'(p)}{2P(p)}dp. $$

Reorganizing, we obtain:

$$-\frac{4qru}{(u^2-1)^2}du = d\mu - \mu f(p)dp, $$

where $f(p) = \frac{P'(p)}{2P(p)}$.

In [173]:
mu = var('mu')  # represents q*r*(u^2 + 1)/(u^2 -1)
dict_vars['mu'] = mu

In [174]:
m = q*r*(u^2 + 1)/(u^2 -1)
diff(m, u).simplify_rational().factor()  # Note this is almost identical to the first term of omega!

-4*q*r*u/((u + 1)^2*(u - 1)^2)

In [176]:
f = (dP/2/P).subs(x==p).simplify_rational().factor()

### $\omega$ in $\mu,p$ variables

In [177]:
mu_keys = ['dk1^dmu', 'dk1^dp', 'dk2^dmu', 'dk2^dp', 'dk1^dk2', 'dmu^dp']
omega_mu = {key: 0 for key in mu_keys}

In [178]:
# Coming from the dk1^du term in omega

coefficient = omega['dk1^du'] / (-4*q*r*u/(u^2-1)^2)
coefficient = coefficient.simplify_rational().factor()
assert diff(coefficient, u) == 0

omega_mu['dk1^dmu'] += coefficient
omega_mu['dk1^dp']  += (-mu * f) * coefficient

In [179]:
# Coming from the dk1^dp term in omega

mu_part = diff(omega['dk1^dp'], q)/r/(u^2+1)*(u^2-1)
mu_part = mu_part.simplify_rational().factor()
assert diff(mu_part, u) == 0

constant_term = omega['dk1^dp'].subs(q==0).simplify_rational().factor()
assert diff(constant_term, u) == 0

omega_mu['dk1^dp'] += mu*mu_part + constant_term

In [180]:
# Coming from the dk2^dp term in omega

omega_mu['dk2^dp'] = omega['dk2^dp']

In [182]:
# Simplify the above terms:
for key in mu_keys:
    try:
        omega_mu[key] = omega_mu[key].simplify_rational().factor()
    except:
        next

# Print omega_mu term by term
for key in mu_keys:
    print '{} term:'.format(key)
    DM(omega_mu[key])
    print ''

dk1^dmu term:


<IPython.core.display.Math object>


dk1^dp term:


<IPython.core.display.Math object>


dk2^dmu term:


...

<IPython.core.display.Math object>


dk1^dk2 term:


<IPython.core.display.Math object>


dmu^dp term:


<IPython.core.display.Math object>